# PLI v0 Demo (Modeling Notebook)

이 노트북은 **가중치 곱셈 기반 PLI(v0)** 데모입니다.  
- 환경적 요인: 날씨(기온)  
- 상황적 요인: 연패, 실책, 대타/대주자, 고의4구  
- 개인적 요인: 컨디션(최근5경기/시즌)  
- **상황 레벨**: 이닝/아웃/주자/점수차 기반 (0~11점 → L1~L4)

> 목표: **WPA × (모든 가중치의 곱)** → PLI 산출  
> WE/WPA, WAR, 파크팩터는 후속 버전에서 붙일 수 있도록 자리만 남겨둠.


In [1]:
# Imports
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, Any
import math
import pandas as pd
import numpy as np
from IPython.display import display
from typing import Optional

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)
wpa = pd.read_csv('../../data/kbo_we_matrix_filtered.csv')
stadium = pd.read_csv('../../data/stadium.csv')
stadium.head()

,stadium,REG_ID,type,city,parkfactor,weather,scode
0,창원NC파크,11H20301,C,창원,1.10,"""nx"": 89, ""ny"": 76",CW
1,수원KT위즈파크,11B20601,C,수원,1.00,"""nx"": 60, ""ny"": 121",SW
2,대구삼성라이온즈파크,11H10701,C,대구,1.50,"""nx"": 90, ""ny"": 90",DG
3,인천SSG랜더스필드,11B20201,C,인천,1.45,"""nx"": 55, ""ny"": 124",MH
4,고척스카이돔,NaN,NaN,서울,0.82,NaN,GC


## 날씨 불러오기

In [45]:
# weather_kma_refactored.py
import requests
import pytz
from datetime import datetime, timedelta
import pandas as pd
import ast  # 문자열 dict 파싱에 사용

# weather 컬럼 → dict로 파싱
stadium["weather"] = stadium["weather"].apply(
    lambda x: ast.literal_eval("{" + x + "}") if isinstance(x, str) and "nx" in x else None
)

# scode → stadium dict 매핑
STADIUM_MAP = {
    row["scode"]: {
        "name": row["stadium"],
        "reg_id": row["REG_ID"],
        "grid": row["weather"],  # {"nx": 89, "ny": 76}
        "parkfactor": row["parkfactor"],
        "city": row["city"],
    }
    for _, row in stadium.iterrows()
}


# 2) 기상청 base_date, base_time 계산
def _kma_base_datetime_now_kst():
    kst = pytz.timezone("Asia/Seoul")
    now = datetime.now(kst)

    if now.minute < 45:
        now = now - timedelta(hours=1)

    base_time_map = {2:'0200',5:'0500',8:'0800',11:'1100',14:'1400',
                     17:'1700',20:'2000',23:'2300'}
    hours = [h for h in base_time_map if h <= now.hour]
    if not hours:
        now = now - timedelta(days=1)
        base_time = '2300'
    else:
        base_time = base_time_map[max(hours)]

    base_date = now.strftime('%Y%m%d')
    return base_date, base_time


# 3) 단일 구장 TMP 조회
def get_stadium_temperature(scode: str, target_hour: int, *, auth_key: str, timeout: int = 10):
    """
    scode: stadium.csv의 scode (예: "CW", "DG")
    target_hour: 0~23 (예: 18 → 18시 예보)
    """
    info = STADIUM_MAP.get(scode.upper())
    if not info:
        raise ValueError(f"Unknown stadium scode: {scode}")

    grid = info["grid"]
    if not grid:
        raise KeyError(f"GRID 좌표 없음: {scode} ({info['name']})")

    base_date, base_time = _kma_base_datetime_now_kst()
    target_time_str = f"{target_hour:02d}00"

    params = {
        "authKey": auth_key,
        "pageNo": "1",
        "numOfRows": "1000",
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": str(grid["nx"]),
        "ny": str(grid["ny"]),
    }
    url = "https://apihub.kma.go.kr/api/typ02/openApi/VilageFcstInfoService_2.0/getVilageFcst"

    try:
        r = requests.get(url, params=params, timeout=timeout)
        r.raise_for_status()
        data = r.json()
        items = data["response"]["body"]["items"]["item"]

        tmp_val = None
        for it in items:
            if it.get("category") == "TMP" and it.get("fcstTime") == target_time_str:
                try:
                    tmp_val = float(it.get("fcstValue"))
                except (TypeError, ValueError):
                    tmp_val = None
                break

        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": tmp_val,
        }

    except Exception as e:
        return {
            "scode": scode,
            "name": info["name"],
            "target_time": target_time_str,
            "temperature": None,
            "error": str(e),
        }


# 4) (선택) 타격 가중치
def batting_weather_weight(temp_c: float | None, base_temp: float = 22.0, alpha: float = 0.02) -> float:
    if temp_c is None or temp_c <= base_temp:
        return 1.0
    return 1.0 + (temp_c - base_temp) * alpha

### 사용 예시

In [46]:
# 비밀키는 안전한 곳에서 꺼내서 넘겨줘!
AUTH = "iVHg1Rm_SGuR4NUZv3hrLw"

# 코드로
res = get_stadium_temperature("JS", target_hour=18, auth_key=AUTH)

# 가중치까지
w = batting_weather_weight(res["temperature"])
print("weather weight =", w)

weather weight = 1.1400000000000001


## 경기장 파크팩터

In [76]:
STADIUM_PARK = dict(zip(stadium["scode"], stadium["parkfactor"]))

def parkfactor_weight(parkfactor: float | None, gamma: float = 0.5,
                      cap_low: float = 0.85, cap_high: float = 1.25) -> float:
    """
    parkfactor ~ 1.00: 중립. 높을수록 타자친화.
    gamma: 민감도 계수 (0.5 → 루트 반영)
    """
    if parkfactor is None:
        return 1.0
    w = 1.0 + gamma * (float(parkfactor) - 1.0)
    return max(min(w, cap_high), cap_low)

## 시즌 기록 가져오기

In [2]:
import os, pathlib, sys
print("CWD:", os.getcwd())
print("sys.path head:", sys.path[:3])

CWD: /Users/yubin/Desktop/2025/dev/Back/ballrae_backend/pli_api
sys.path head: ['/app', '/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/opt/homebrew/Cellar/python@3.12/3.12.10/Frameworks/Python.framework/Versions/3.12/lib/python3.12']


In [3]:
import os, sys, pathlib
PROJ_ROOT = pathlib.Path("/Users/yubin/Desktop/2025/dev/Back")
sys.path.insert(0, str(PROJ_ROOT))

# 1) 먼저 .env 로드
try:
    from dotenv import load_dotenv
    load_dotenv(PROJ_ROOT / ".env")
except Exception:
    pass

# 2) 그 다음에 DB_HOST 덮어쓰기 (중요!)
os.environ["DB_HOST"] = "127.0.0.1"
os.environ["DJANGO_SETTINGS_MODULE"] = "ballrae_backend.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

# 확인
from django.conf import settings
print("DB HOST:", settings.DATABASES["default"]["HOST"])

DB HOST: 127.0.0.1


In [14]:
from ballrae_backend.players.models import Batter
batter = Batter.objects.filter(id="1377").first()

print(batter)

Batter object (1377)


In [23]:
from ballrae_backend.games.models import Player
Player.objects.all().values()

<QuerySet [{'id': 6247, 'player_name': '소형준', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '50030'}, {'id': 6248, 'player_name': '박영현', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '52060'}, {'id': 6249, 'player_name': '원상현', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '54063'}, {'id': 6250, 'player_name': '고영표', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '64001'}, {'id': 6251, 'player_name': '주권', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '65060'}, {'id': 6252, 'player_name': '이상동', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '69054'}, {'id': 6253, 'player_name': '우규민', 'position': 'P', 'team_id': 'KT', 'player_bdate': None, 'pcode': '73117'}, {'id': 6254, 'player_name': '장성우', 'position': 'B', 'team_id': 'KT', 'player_bdate': None, 'pcode': '78548'}, {'id': 6255, 'player_name': '김기중', 'position': 'P', 'team_id': 'HH', 'player_bdate': None, 'pcode': '51715'}, 

In [13]:
from ballrae_backend.players.models import BatterRecent
BatterRecent.objects.all().values()

<QuerySet [{'id': 3, 'batter_id': 1377, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 45, 210244)}, {'id': 2, 'batter_id': 1376, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 44, 504535)}, {'id': 5, 'batter_id': 1379, 'ab': 1, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 46, 549991)}, {'id': 6, 'batter_id': 1380, 'ab': 3, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 47, 164076)}, {'id': 7, 'batter_id': 1381, 'ab': 0, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 47, 829847)}, {'id': 8, 'batter_id': 1382, 'ab': 6, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 52, 756523)}, {'id': 9, 'batter_id': 1383, 'ab': 10, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 53, 346451)}, {'id': 10, 'batter_id': 1384, 'ab': 6, 'hits': 0, 'updated_at': datetime.datetime(2025, 8, 23, 16, 48, 53, 941969)}, {'id': 11, 'batter_id': 1385, 'ab': 12, 'hits': 0, 'updated

In [15]:
from ballrae_backend.players.models import BatterRecent
batterRecent = BatterRecent.objects.filter(batter=batter).first()

print(batterRecent)

BatterRecent object (3)


## 컨디션 지수

In [29]:
# 컨디션 지수 계산
def calculate_condition(pcode: str, season: int = 2025):
    from ballrae_backend.games.models import Player
    from ballrae_backend.players.models import Batter, Pitcher, BatterRecent
    
    try:
        player = Player.objects.get(pcode=pcode)
        
        if player.position == 'B':  # 타자
            try:
                batter = Batter.objects.get(player=player, season=season)
                recent = BatterRecent.objects.get(batter=batter)
                # 타율 계산
                ab = batter.ab or 0
                hits = (batter.singles or 0) + (batter.doubles or 0) + (batter.triples or 0) + (batter.homeruns or 0)
                
                condition = (recent.hits/recent.ab)/(hits/ab)
                result = 1.0 + (condition - 1.0) * 0.5
                return result

            except Batter.DoesNotExist:
                return None
                
        elif player.position == 'P':  # 투수
            return None
                
    except Player.DoesNotExist:
        return None
    
    except ZeroDivisionError:
        return 0.0

In [31]:
calculate_condition(65703)

0.5

## 가중치 함수 (v0)

In [43]:
def streak_weight(loss_streak: Optional[int]) -> float:
    if loss_streak is None or loss_streak <= 2: return 1.0
    if loss_streak <= 4: return 0.95
    if loss_streak <= 6: return 0.90
    return 0.85

def pinch_weight(flag: bool) -> float:
    return 1.8 if flag else 1.0

def ibb_focus_weight(flag: bool) -> float:
    return 2.0 if flag else 1.0

def error_momentum_weight(flag: bool, next_batter: bool=False) -> float:
    if not flag: return 1.0
    return 1.1 if next_batter else 1.3

def combine_weights(*ws: float) -> float:
    v = 1.0
    for w in ws:
        if w and w > 0:
            v *= float(w)
    return v


## 상황 레벨 (0~11점 → L1~L4)

In [42]:
def calculate_situation_level(inning: int, outs: int, runners_code: str, score_diff: int):
    total = 0
    # 이닝
    if inning >= 9: total += 3
    elif inning == 8: total += 2
    elif inning == 7: total += 1
    # 아웃
    if outs == 0: total += 2
    elif outs == 1: total += 1
    # 주자
    runner_scores = {
        '000':0,'100':1,'010':2,'110':2,'001':3,'101':3,'011':3,'111':3
    }
    total += runner_scores.get(runners_code, 0)
    # 점수차(절대값)
    d = abs(score_diff)
    if d == 0: total += 3
    elif d == 1: total += 2
    elif d <= 3: total += 1
    # 레벨
    if total >= 9: return 4, "최대 위기/방화", total
    if total >= 6: return 3, "중요한 승부처", total
    if total >= 3: return 2, "관리 필요", total
    return 1, "일상적 교체", total


## PLI 계산기

In [62]:
def compute_pli_row(row: pd.Series, base_wpa: float) -> Dict[str, Any]:
    # 상황 레벨
    level, desc, score = calculate_situation_level(
        int(row['inning']), int(row['outs']), str(row['runners_code']), int(row['score_diff'])
    )
    # 가중치
    w_env = batting_weather_weight(row.get('temp_c'))
    w_situ = combine_weights(
        streak_weight(row.get('loss_streak')),
        pinch_weight(bool(row.get('pinch_event', False))),
        ibb_focus_weight(bool(row.get('intentional_walk', False))),
        error_momentum_weight(bool(row.get('error_happened', False)), bool(row.get('next_batter_after_error', False)))
    )
    pf = STADIUM_PARK.get('scode')
    w_park = parkfactor_weight(pf)
    w_total = combine_weights(w_env, w_situ, w_park)
    pli = base_wpa * w_total
    return {
        "pli": round(pli, 4),
        "w_env": round(w_env, 4),
        "w_situ": round(w_situ, 4),
        "w_total": round(w_total, 4),
        "level": level, "level_desc": desc, "level_score": score
    }


## WPA

In [58]:
# half를 'top'과 'bot'으로 구분하기 위해 새로운 컬럼 생성
wpa['is_home_offense'] = wpa['half'] == 'bot'

In [60]:
WE_MAP = {}
for _, row in wpa.iterrows():
    # 튜플 키: (inning, is_home_offense, outs, base1, base2, base3, score_diff)
    key = (
        row['inning_number'],
        row['is_home_offense'],
        row['out'],
        row['runner_on_1b'],
        row['runner_on_2b'],
        row['runner_on_3b'],
        row['score_diff']
    )
    WE_MAP[key] = row['win_expectancy']

In [102]:
def get_win_expectancy(inning, half, outs, runners_code, score_diff):
    is_home = (half == 'bot')
    
    # 주자 코드 '101'을 (1, 0, 1)로 변환
    runners = [int(r) for r in runners_code]
    
    # key 생성
    key = (inning, is_home, outs, runners[0], runners[1], runners[2], score_diff)
    
    # 딕셔너리에서 값 찾기
    return WE_MAP.get(key, None) # 해당 상황이 없으면 None 반환

def get_base_wpa(atbat_start, atbat_end):
    # 타석 시작 시점의 WE 계산
    we_start = get_win_expectancy(
        atbat_start['inning'],
        atbat_start['half'],
        atbat_start['outs'],
        atbat_start['runners_code'],
        atbat_start['score_diff']
    )
    
    # 타석 종료 시점의 WE 계산
    we_end = get_win_expectancy(
        atbat_end['inning'],
        atbat_end['half'],
        atbat_end['outs'],
        atbat_end['runners_code'],
        atbat_end['score_diff']
    )
    
    # === 디버깅용 로그 추가 ===
    # print(f"Start Situation: inning={atbat_start['inning']}, half={atbat_start['half']}, outs={atbat_start['outs']}, runners_code={atbat_start['runners_code']}, score_diff={atbat_start['score_diff']}")
    # print(f"End Situation:   inning={atbat_end['inning']}, half={atbat_end['half']}, outs={atbat_end['outs']}, runners_code={atbat_end['runners_code']}, score_diff={atbat_end['score_diff']}")
    print(f"WE Start: {we_start}, WE End: {we_end}")
    
    # WE를 찾지 못하면 기본값 (0.12)을 사용
    if we_start is None or we_end is None:
        print("Warning: WE data not found, returning default WPA (0.12).")
        return 0.12
    
    return we_end - we_start

In [106]:
# adapters/pli_adapter.py
import re
from typing import Dict, Any, List, Optional

KW_PINCH = ("대타", "대주자")
KW_IBB = ("고의")
KW_ERROR = ("실책",)

def runners_code_from_onbase(on_base: Dict[str, str]) -> str:
    """
    on_base 예: {"base1":"0","base2":"9","base3":"0"}
    '0'이 아니면 1(점유)로 본다.
    반환: 'XYZ' (1루-2루-3루) → '101' 등
    """
    def occ(v: Optional[str]) -> str:
        return "1" if (v is not None and str(v) != "0") else "0"
    b1 = occ(on_base.get("base1"))
    b2 = occ(on_base.get("base2"))
    b3 = occ(on_base.get("base3"))
    return f"{b1}{b2}{b3}"

def parse_score_pair(score: str):
    a, b = score.split(":")
    a, b = int(a), int(b)
    return b, a

def batting_team_for_half(half: str, home_team: str, away_team: str) -> str:
    return away_team if half == "top" else home_team

def score_diff_for_batting(score: str, half: str, home_team: str, away_team: str) -> int:
    """
    타자(공격) 팀 기준 점수차 = (타자 팀 득점 - 상대 득점)
    """
    home, away = parse_score_pair(score)
    batting_team = batting_team_for_half(half, home_team, away_team)
    if batting_team == home_team:
        return home - away
    else:
        return away - home

def text_has_any(s: Optional[str], keywords: tuple) -> bool:
    if not s:
        return False
    return any(kw in s for kw in keywords)

def detect_triggers(atbat: Dict[str, Any]) -> Dict[str, bool]:
    pinch = False
    ibb = False
    error = False

    # 1) 대타/대주자
    if atbat.get("original_batter") and atbat.get("actual_batter"):
        if atbat["original_batter"] != atbat["actual_batter"]:
            pinch = True
    # 텍스트 키워드
    if text_has_any(atbat.get("full_result"), KW_PINCH) or text_has_any(atbat.get("main_result"), KW_PINCH):
        pinch = True
    # pitch_sequence 중 event 텍스트도 훑기
    for ps in atbat.get("pitch_sequence", []):
        ev = ps.get("event")
        if isinstance(ev, str):
            if text_has_any(ev, KW_PINCH):
                pinch = True
        elif isinstance(ev, list):
            if any(text_has_any(x, KW_PINCH) for x in ev if isinstance(x, str)):
                pinch = True

    # 2) 고의4구
    if text_has_any(atbat.get("full_result"), KW_IBB) or text_has_any(atbat.get("main_result"), KW_IBB):
        ibb = True

    # 3) 실책
    if text_has_any(atbat.get("full_result"), KW_ERROR) or text_has_any(atbat.get("main_result"), KW_ERROR):
        error = True

    return {
        "pinch_event": pinch,
        "intentional_walk": ibb,
        "error_happened": error,
    }

def atbat_to_pli_payload(
    atbat: Dict[str, Any],
    context: Dict[str, Any],
    *,
    temp_c: Optional[float] = None,
    base_wpa: Optional[float] = None,
    loss_streak: Optional[int] = None,
    score_format: str = "home:away",
) -> Dict[str, Any]:
    inning = int(atbat["inning"])
    condition = calculate_condition(atbat.get("actual_batter"))
    outs = int(atbat["out"])
    half = atbat.get("half", "top")
    runners_code = runners_code_from_onbase(atbat.get("on_base", {}))
    score_str = atbat.get("score", "0:0")

    score_diff = score_diff_for_batting(
        score_str, half, context["home_team"], context["away_team"]
    )
    triggers = detect_triggers(atbat)
    bwpa = float(base_wpa if base_wpa is not None else 0.12)

    return {
        # === 식별자(타자/투수/게임/타석) 유지 ===
        "game_id": context.get("game_id"),
        "batter_id": atbat.get("actual_batter"),
        "pitcher_id": atbat.get("pitcher"),
        "inning": inning,
        "half": half,
        "bat_order": atbat.get("bat_order"),
        # === 상황 입력 ===
        "outs": outs,
        "runners_code": runners_code,
        "score_diff": score_diff,
        "base_wpa": bwpa,
        # === 외부 요인(초기값 None이면 weight=1.0) ===
        "temp_c": temp_c,
        "loss_streak": loss_streak,
        "condition":condition,
        # === 트리거 ===
        "pinch_event": triggers["pinch_event"],
        "intentional_walk": triggers["intentional_walk"],
        "error_happened": triggers["error_happened"],
        "next_batter_after_error": False,
    }

def inning_pack_to_pli_requests(feed: Dict[str, Any], *, temp_c: Optional[float] = None, score_format: str = "home:away") -> List[Dict[str, Any]]:
    data = feed["data"]
    ctx = {
        "home_team": data["defense_positions"]["home_team"],
        "away_team": data["defense_positions"]["away_team"],
    }
    requests: List[Dict[str, Any]] = []

    def _absorb(side: str):
        side_block = data.get(side)
        if not side_block:
            return
        inning_default = side_block.get("inning_number")
        atbats = side_block.get("atbats", [])

        for idx, ab in enumerate(atbats, start=1):
            # half 기본값: side(top/bot)
            ab_half = ab.get("half") or side
            # bat_order 기본값: 원본 bat_order → 없으면 appearance_number → 없으면 enumerate 인덱스
            ab_bat_order = ab.get("bat_order")
            if ab_bat_order in (None, ""):
                ab_bat_order = ab.get("appearance_number") or idx

            # inning 기본값: 각 atbat의 inning → 없으면 side_block의 inning_number
            ab_inning = ab.get("inning") or inning_default

            # 식별자 주입
            ab = {
                **ab,
                "half": ab_half,
                "bat_order": ab_bat_order,
                "inning": ab_inning,
            }

            req = atbat_to_pli_payload(
                ab, ctx, temp_c=temp_c, score_format=score_format
            )
            requests.append(req)

    _absorb("top")
    _absorb("bot")
    return requests

def inning_pack_to_pli_scores(
    feed: dict,
    *,
    temp_c: float | None = None,
    loss_streak: int | None = None,
    score_format: str = "home:away",
    return_detail: bool = False,
):
    """
    WE(승리 기대치)를 기반으로 PLI를 계산하여 반환합니다.
    """
    reqs = inning_pack_to_pli_requests(feed, temp_c=temp_c, score_format=score_format)
    results = []
    
    # 마지막 타석은 다음 타석이 없으므로 WPA 계산이 불가능해서 제외.
    # 하지만 WE 기반 PLI는 각 타석의 종료 시점 데이터만 사용하므로
    # 마지막 타석까지 계산할 수 있습니다.
    for r in reqs:
        # 1. 타석 종료 시점의 WE 값 가져오기
        # 이닝, 아웃, 주자 상황, 점수차를 바탕으로 WE 값을 찾습니다.
        # 만약 WE 데이터가 없으면 기본값인 0.5 (50% 확률)을 사용합니다.
        we_end = get_win_expectancy(
            r['inning'],
            r['half'],
            r['outs'],
            r['runners_code'],
            r['score_diff']
        )
        base_we = we_end if we_end is not None else 0.5

        # 2. 모든 가중치 요소 계산
        level, desc, total_score = calculate_situation_level(
            r["inning"], r["outs"], r["runners_code"], r["score_diff"]
        )
        pcode = r.get('batter_id')
        
        w_env = batting_weather_weight(r.get("temp_c"))
        w_personal = combine_weights(
            calculate_condition(pcode, 2025),
        )
        w_situ = combine_weights(
            streak_weight(r.get("loss_streak")),
            pinch_weight(bool(r.get("pinch_event"))),
            ibb_focus_weight(bool(r.get("intentional_walk"))),
            error_momentum_weight(bool(r.get("error_happened")), bool(r.get("next_batter_after_error")))
        )
        w_total = combine_weights(w_env, w_personal, w_situ)
        
        # 3. 최종 PLI 값 계산 (WE 값에 가중치를 곱함)
        pli_val = round(float(base_we) * w_total, 4)

        if return_detail:
            results.append({
                "input": r,
                "pli": pli_val,
                "weights": {
                    "environment": round(w_env, 4),
                    "personal": round(w_personal, 4),
                    "situational": round(w_situ, 4),
                    "combined": round(w_total, 4),
                },
                "situation": {"level": level, "description": desc, "score": total_score},
            })
        else:
            results.append({
                "inning": r["inning"],
                "half": r.get("half"),
                "bat_order": r.get("bat_order"),
                "pli": pli_val,
            })
            
    return results

## 더미데이터

In [107]:
import json

raw_data = json.loads("""

{
    "status": "OK_REALTIME",
    "message": "8회 이닝 정보 (실시간)",
    "data": {
        "top": {
            "game": "20250828LGNC02025",
            "inning_number": 8,
            "atbats": [
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6865,
                        "pcode": "67954",
                        "player_name": "김진호"
                    },
                    "bat_order": 5,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7095,
                        "pcode": "79109",
                        "player_name": "오지환"
                    },
                    "out": "0",
                    "score": "3:3",
                    "on_base": {
                        "base1": "5",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.34,
                        1.62,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "체인지업",
                            "pitch_coordinate": [
                                [
                                    -1.0680164653841242,
                                    3.71162868300673
                                ]
                            ],
                            "speed": "128",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": "대주자 최정원 : 3루수 서호철 (으)로 교체"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.10825995745153083,
                                    4.122701705891309
                                ]
                            ],
                            "speed": "147",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.28360043121826384,
                                    2.030983350487592
                                ]
                            ],
                            "speed": "144",
                            "count": "2-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "우익수 앞 1루타"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6865,
                        "pcode": "67954",
                        "player_name": "김진호"
                    },
                    "bat_order": 6,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7099,
                        "pcode": "69100",
                        "player_name": "구본혁"
                    },
                    "out": "0",
                    "score": "3:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.168,
                        1.537,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 오지환 : 2루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.14813454490427858,
                                    2.6618541115977363
                                ]
                            ],
                            "speed": "147",
                            "count": "0-1",
                            "pitch_result": "번트파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.7535793099931689,
                                    3.16299331059831
                                ]
                            ],
                            "speed": "146",
                            "count": "0-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "1루수 희생번트 아웃 (1루수 태그아웃)"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6865,
                        "pcode": "67954",
                        "player_name": "김진호"
                    },
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7086,
                        "pcode": "55164",
                        "player_name": "박관우"
                    },
                    "out": "1",
                    "score": "3:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": null,
                    "full_result": "7번타자 최원영 : 대타 박관우 (으)로 교체",
                    "pitch_sequence": []
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6865,
                        "pcode": "67954",
                        "player_name": "김진호"
                    },
                    "bat_order": 7,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7086,
                        "pcode": "55164",
                        "player_name": "박관우"
                    },
                    "out": "1",
                    "score": "3:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "5",
                        "base3": "0"
                    },
                    "appearance_number": 2,
                    "strike_zone": [
                        3.195,
                        1.55,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "체인지업",
                            "pitch_coordinate": [
                                [
                                    -0.690665768460665,
                                    3.0037190538700096
                                ]
                            ],
                            "speed": "127",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "체인지업",
                            "pitch_coordinate": [
                                [
                                    1.0744643735216992,
                                    0.3811444018816008
                                ]
                            ],
                            "speed": "127",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.01631849096835336,
                                    3.5097664971657547
                                ]
                            ],
                            "speed": "145",
                            "count": "1-2",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "체인지업",
                            "pitch_coordinate": [
                                [
                                    -1.260408805698227,
                                    2.698599290061372
                                ]
                            ],
                            "speed": "130",
                            "count": "1-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌익수 플라이 아웃"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6865,
                        "pcode": "67954",
                        "player_name": "김진호"
                    },
                    "bat_order": 8,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7097,
                        "pcode": "79365",
                        "player_name": "박동원"
                    },
                    "out": "2",
                    "score": "4:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.314,
                        1.607,
                        0.75,
                        -0.75
                    ],
                    "full_result": "2루주자 오지환 : 홈인",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.4684822136320576,
                                    3.63414260782277
                                ]
                            ],
                            "speed": "128",
                            "count": "0-1",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.8553030500353362,
                                    1.9117431707642416
                                ]
                            ],
                            "speed": "146",
                            "count": "0-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.29673200523509724,
                                    4.632000385422626
                                ]
                            ],
                            "speed": "148",
                            "count": "1-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "체인지업",
                            "pitch_coordinate": [
                                [
                                    1.3820278285611811,
                                    1.32231462386769
                                ]
                            ],
                            "speed": "131",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.6322377061230803,
                                    3.7690115678046925
                                ]
                            ],
                            "speed": "128",
                            "count": "2-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌중간 2루타"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6752,
                        "pcode": "54904",
                        "player_name": "손주환"
                    },
                    "bat_order": 9,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7087,
                        "pcode": "62415",
                        "player_name": "박해민"
                    },
                    "out": "2",
                    "score": "4:3",
                    "on_base": {
                        "base1": "9",
                        "base2": "8",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.338,
                        1.619,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -3.0011673169105904,
                                    4.391213305378095
                                ]
                            ],
                            "speed": "147",
                            "count": "1-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -1.162499206328478,
                                    3.256558596399532
                                ]
                            ],
                            "speed": "134",
                            "count": "2-0",
                            "pitch_result": "볼",
                            "event": "투수 김진호 : 투수 손주환 (으)로 교체"
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.5205327794738015,
                                    2.6222480210552117
                                ]
                            ],
                            "speed": "146",
                            "count": "3-0",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.1290757595662806,
                                    3.50020961396903
                                ]
                            ],
                            "speed": "145",
                            "count": "4-0",
                            "pitch_result": "볼",
                            "event": null
                        }
                    ],
                    "main_result": "볼넷"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6752,
                        "pcode": "54904",
                        "player_name": "손주환"
                    },
                    "bat_order": 1,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7100,
                        "pcode": "65207",
                        "player_name": "신민재"
                    },
                    "out": "2",
                    "score": "4:3",
                    "on_base": {
                        "base1": "1",
                        "base2": "9",
                        "base3": "8"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.133,
                        1.52,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 박해민 : 2루까지 진루|2루주자 박동원 : 3루까지 진루",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.26898731034699097,
                                    2.1976842926127134
                                ]
                            ],
                            "speed": "146",
                            "count": "0-0",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "중견수 앞 1루타"
                },
                {
                    "inning": 8,
                    "half": "top",
                    "pitcher": {
                        "id": 6752,
                        "pcode": "54904",
                        "player_name": "손주환"
                    },
                    "bat_order": 2,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 7066,
                        "pcode": "68119",
                        "player_name": "문성주"
                    },
                    "out": "2",
                    "score": "4:3",
                    "on_base": {
                        "base1": "1",
                        "base2": "9",
                        "base3": "8"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.186,
                        1.545,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    -0.6025482561717881,
                                    2.541025345812121
                                ]
                            ],
                            "speed": "134",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    1.0066777470619725,
                                    2.3341593107619065
                                ]
                            ],
                            "speed": "147",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "슬라이더",
                            "pitch_coordinate": [
                                [
                                    0.9388404712885063,
                                    2.062159503484431
                                ]
                            ],
                            "speed": "134",
                            "count": "2-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -1.0999083491608364,
                                    1.7723859109312001
                                ]
                            ],
                            "speed": "146",
                            "count": "3-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.6388188281040624,
                                    2.7380117633807273
                                ]
                            ],
                            "speed": "146",
                            "count": "3-1",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "2루수 땅볼 아웃 (2루수->1루수 송구아웃)"
                }
            ]
        },
        "bot": {
            "game": "20250828LGNC02025",
            "inning_number": 8,
            "atbats": [
                {
                    "inning": 8,
                    "half": "bot",
                    "pitcher": {
                        "id": 7048,
                        "pcode": "75867",
                        "player_name": "김진성"
                    },
                    "bat_order": 2,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 6464,
                        "pcode": "66606",
                        "player_name": "최원준"
                    },
                    "out": "0",
                    "score": "4:3",
                    "on_base": {
                        "base1": "2",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.276,
                        1.589,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.613382877895507,
                                    3.0963118352465173
                                ]
                            ],
                            "speed": "141",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": "투수 김영우 : 투수 김진성 (으)로 교체"
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.3943998215546336,
                                    3.208040761474522
                                ]
                            ],
                            "speed": "143",
                            "count": "0-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.527972970853408,
                                    3.5793674649246974
                                ]
                            ],
                            "speed": "143",
                            "count": "0-2",
                            "pitch_result": "타격",
                            "event": null
                        }
                    ],
                    "main_result": "좌중간 1루타"
                },
                {
                    "inning": 8,
                    "half": "bot",
                    "pitcher": {
                        "id": 7048,
                        "pcode": "75867",
                        "player_name": "김진성"
                    },
                    "bat_order": 3,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 6871,
                        "pcode": "62907",
                        "player_name": "박민우"
                    },
                    "out": "0",
                    "score": "4:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.364,
                        1.631,
                        0.75,
                        -0.75
                    ],
                    "full_result": "1루주자 최원준 : 도루실패아웃 (포수->유격수 태그아웃)",
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.07049317563759283,
                                    1.8907115890974557
                                ]
                            ],
                            "speed": "125",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.159567598712257,
                                    3.5698511518763842
                                ]
                            ],
                            "speed": "142",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": "투수 투수판 이탈"
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.2953284696141454,
                                    1.6163290226663831
                                ]
                            ],
                            "speed": "141",
                            "count": "1-2",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.21749693810808346,
                                    1.621305848210051
                                ]
                            ],
                            "speed": "126",
                            "count": "1-2",
                            "pitch_result": "파울",
                            "event": "투수 투수판 이탈|투수 투수판 이탈"
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.45976994871799615,
                                    3.9778601805529723
                                ]
                            ],
                            "speed": "126",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 6,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.5850610112561098,
                                    2.85319715820526
                                ]
                            ],
                            "speed": "142",
                            "count": "2-2",
                            "pitch_result": "파울",
                            "event": null
                        },
                        {
                            "pitch_num": 7,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    0.11742759693546767,
                                    3.3137747431532976
                                ]
                            ],
                            "speed": "127",
                            "count": "2-3",
                            "pitch_result": "헛스윙",
                            "event": "투수 투수판 이탈"
                        },
                        {
                            "pitch_num": 8,
                            "event": [
                                "1루주자 최원준 : 도루실패아웃 (포수->유격수 태그아웃)"
                            ]
                        }
                    ],
                    "main_result": "삼진 아웃"
                },
                {
                    "inning": 8,
                    "half": "bot",
                    "pitcher": {
                        "id": 7048,
                        "pcode": "75867",
                        "player_name": "김진성"
                    },
                    "bat_order": 4,
                    "original_batter": null,
                    "actual_batter": {
                        "id": 6878,
                        "pcode": "54944",
                        "player_name": "데이비슨"
                    },
                    "out": "2",
                    "score": "4:3",
                    "on_base": {
                        "base1": "0",
                        "base2": "0",
                        "base3": "0"
                    },
                    "appearance_number": 1,
                    "strike_zone": [
                        3.503,
                        1.699,
                        0.75,
                        -0.75
                    ],
                    "full_result": null,
                    "pitch_sequence": [
                        {
                            "pitch_num": 1,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.499788195657057,
                                    3.333385057544423
                                ]
                            ],
                            "speed": "127",
                            "count": "0-1",
                            "pitch_result": "스트라이크",
                            "event": null
                        },
                        {
                            "pitch_num": 2,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    0.8055917503163212,
                                    1.4058204884843835
                                ]
                            ],
                            "speed": "144",
                            "count": "1-1",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 3,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.2294174483791913,
                                    1.5089496629743486
                                ]
                            ],
                            "speed": "127",
                            "count": "1-2",
                            "pitch_result": "헛스윙",
                            "event": null
                        },
                        {
                            "pitch_num": 4,
                            "pitch_type": "포크",
                            "pitch_coordinate": [
                                [
                                    -0.3622586216320855,
                                    0.7907675279571054
                                ]
                            ],
                            "speed": "128",
                            "count": "2-2",
                            "pitch_result": "볼",
                            "event": null
                        },
                        {
                            "pitch_num": 5,
                            "pitch_type": "직구",
                            "pitch_coordinate": [
                                [
                                    -0.256512246501558,
                                    1.9388268328863911
                                ]
                            ],
                            "speed": "144",
                            "count": "2-3",
                            "pitch_result": "스트라이크",
                            "event": null
                        }
                    ],
                    "main_result": "삼진 아웃"
                }
            ]
        },
        "defense_positions": {
            "home_team": "NC",
            "away_team": "LG",
            "home": {
                "유격수": "김주원",
                "중견수": "최원준",
                "2루수": "박민우",
                "1루수": "데이비슨",
                "지명타자": "박건우",
                "좌익수": "이우성",
                "포수": "김형준",
                "3루수": "김휘집",
                "11": "최정원",
                "우익수": "천재환",
                "투수": "손주환"
            },
            "away": {
                "지명타자": "김현수",
                "우익수": "문성주",
                "1루수": "오스틴",
                "3루수": "문보경",
                "유격수": "오지환",
                "2루수": "구본혁",
                "좌익수": "천성호",
                "포수": "이주헌",
                "중견수": "박해민",
                "투수": "유영찬"
            }
        }
    }
}""")

## test

In [108]:
results = inning_pack_to_pli_scores(raw_data)

for r in results:
    print(r)

{'inning': 8, 'half': 'top', 'bat_order': 5, 'pli': 0.4459}
{'inning': 8, 'half': 'top', 'bat_order': 6, 'pli': 0.1875}
{'inning': 8, 'half': 'top', 'bat_order': 7, 'pli': 0.3493}
{'inning': 8, 'half': 'top', 'bat_order': 7, 'pli': 0.194}
{'inning': 8, 'half': 'top', 'bat_order': 8, 'pli': 0.7857}
{'inning': 8, 'half': 'top', 'bat_order': 9, 'pli': 0.7963}
{'inning': 8, 'half': 'top', 'bat_order': 1, 'pli': 0.6071}
{'inning': 8, 'half': 'top', 'bat_order': 2, 'pli': 0.6071}
{'inning': 8, 'half': 'bot', 'bat_order': 2, 'pli': 0.2841}
{'inning': 8, 'half': 'bot', 'bat_order': 3, 'pli': 0.2131}
{'inning': 8, 'half': 'bot', 'bat_order': 4, 'pli': 0.0897}
